In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install accelerate -U
!pip install torch
!pip install bitsandbytes


In [ ]:
import pandas as pd

data = pd.read_csv('drive/MyDrive/processed_data.tsv', delimiter='\t', encoding='utf-8')


In [ ]:
data['input_text'] = 'detoxify: ' + data['reference']
data['target_text'] = data['translation']


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split


In [ ]:
# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1)

# Convert the train and validation datasets to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('google/t5-efficient-tiny')
model = T5ForConditionalGeneration.from_pretrained('google/t5-efficient-tiny')

# Tokenize the datasets
def tokenize(batch):
    tokenized_input = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
    tokenized_target = tokenizer(batch['target_text'], padding='max_length', truncation=True, max_length=512)
    tokenized_input['labels'] = tokenized_target['input_ids']
    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/519999 [00:00<?, ? examples/s]

Map:   0%|          | 0/57778 [00:00<?, ? examples/s]

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# Save the model
model.save_pretrained('my_detoxified_t5_model')
tokenizer.save_pretrained('my_detoxified_t5_model')


Step,Training Loss
10,24.055400
20,24.159200
30,23.316800
40,23.535400
50,22.757000
60,22.701000
70,21.827100
80,21.374400
90,20.672200
100,20.005800


KeyboardInterrupt: ignored

In [ ]:
import transformers
ModelTokenizer = transformers.models.t5.tokenization_t5_fast.T5TokenizerFast
def inference(text_model, inference_request: str, tokenizer) -> str:
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = text_model.generate(input_ids=input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True, temperature=0)

In [ ]:
model2=model.cuda()

In [ ]:
inference(model2.cpu(), "Fuck it I'm done with this shit goddamn", tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'shit, shit, shit, shit.'